# 1. Tabular dataset -> Graph dataset

Example of how to convert a CSV file into a graph dataset 


`Step 0`

Working with data requires some creativity. It is also where inspiration for a project degrades into work! It's normal to take some time to rearrange the data in a graph format. 

This notebook is only intended to get you started. It still needs to be mapped to a specific use case.

`Step 1`

Identify the following items in your dataset:

- Nodes (Items, People, Locations, Cars, ...)
- Edges (Connections, Interactions, Similarity, ...)
- Node Features (Attributes)
- Labels (Node-level, edge-level, graph-level)

and optionally:
- Edge weights (Strength of the connection, number of interactions, ...)
- Edge features (Additional (multi-dim) properties describing the edge)



`Step 2`

Do you have different node and edge types? (This means the nodes/edges have different attributes such as Cars vs. People)

- No, all my edges/nodes have the same type  --> **Proceed with 1.1**
- Yes, there are different relations and node types --> **Proceed with 1.2**


## 1.1 Homogeneous

`Example 1 / Step 3`

The following dataset was selected pretty much at random to avoid subjective bias found on the internet that contains homogeneous nodes. 

The dataset is the [FIFA 21 Rating dataset](https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset), a dataset with soccer players.

Below we extract a small subset of the scraped data (there is much more available!) to build a graph dataset out of it. Have a look at the pandas Dataframe below.

In [ ]:
import pandas as pd

# Download data (quietly)
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player_skill.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_team.csv

# Load data
player_df = pd.read_csv("tbl_player.csv")
skill_df = pd.read_csv("tbl_player_skill.csv")
team_df = pd.read_csv("tbl_team.csv")

# Extract subsets
player_df = player_df[["int_player_id", "str_player_name", "str_positions", "int_overall_rating", "int_team_id"]]
skill_df = skill_df[["int_player_id", "int_long_passing", "int_ball_control", "int_dribbling"]]
team_df = team_df[["int_team_id", "str_team_name", "int_overall"]]

# Merge data
player_df = player_df.merge(skill_df, on='int_player_id')
fifa_df = player_df.merge(team_df, on='int_team_id')

# Sort dataframe
fifa_df = fifa_df.sort_values(by="int_overall_rating", ascending=False)
print("Players: ", fifa_df.shape[0])
fifa_df.head()

Identify the graph-specific things we need:

- `Nodes` - Football (soccer) players (by ID)
- `Edges` - If they play for the same team (see explanation below)
- `Node Features` - The football player's position, specialities, ball control, ...
- `Labels` - The football player's overall rating (node-level regression task)

Nodes are usually pretty straight-forward to identify - here we even have IDs. If you don't have a unique identifier, you need one, because you need to know between which nodes a connection exists.

The most challenging task is typically linking nodes with edges. Here we define the edges based on the team assignment. With this dataset, we could predict the expected rating when a player switches to a new team or a new player is observed. Therefore we expect relational effects through the team assignment. Of course there are many other possibilities to define the edges such as:
- How many times two players played together (edge weight), i.e.,  Synergies
- How many times a player has won/lost 1:1 duels (edge weight)
- Started their career in the same football club 
- Temporal edges: "Played together in the last 2 weeks"
- ...

There are many choices for how to combine instances in the dataframe. The choice is application dependent. We will continue with the easiest approach, which is connecting them accoring to their team assignments.



In [ ]:
# Make sure that we have no duplicate nodes
max(fifa_df["int_player_id"].value_counts())

Each football player ID occurs only once in the dataset. 



> Note: We plan to build one single graph here. If individual node-id's occur more than once in your dataset, there are different options:

- You have multiple graphs that can contain the same node. In this case you need to iterate over each subset of your dataframe that belongs to one individual graph and do the calculations on this subset.
- You have to aggregate multiple rows into one. For example if you have transactional data (like a payment history), you would need to summarize this somehow into one feature vector, such as: #payments, payment amount, ...
- If you have a temporal dataset and need to check section 2.)


`Preprocessing one single graph...`

`Step 4`: Extract the node features

The node features are typically represented in a matrix of the shape *(num_nodes, node_feature_dim)*.

For each of the football players, we simply extract their attributes. Because each player id is unique, we can easily do this based on the original dataframe. Look at the examples below to see how an aggregation can look like if you have multiple rows for individual nodes.

In [ ]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
node_features = sorted_df[["str_positions", "int_long_passing", "int_ball_control", "int_dribbling"]]
# Convert non-numeric columns
pd.set_option('mode.chained_assignment', None)
positions = node_features["str_positions"].str.split(",", expand=True)
node_features["first_position"] = positions[0]
# One-hot encoding
node_features = pd.concat([node_features, pd.get_dummies(node_features["first_position"])], axis=1, join='inner')
node_features.drop(["str_positions", "first_position"], axis=1, inplace=True)
node_features.head() 

That's our node feature matrix. The number of nodes and the ordering is implicitly defined by it's shape. Each row corresponds to one node in our final graph. 

In [ ]:
# Convert to numpy
x = node_features.to_numpy()
x.shape # [num_nodes x num_features]

`Step 5`: Extract the labels

The labels will be the ratings of each of the players. This corresponds to a node-level prediction problem. Therefore we have as many labels as we have nodes. Of course it can happen that we don't have labels for all nodes and in this case it makes sense to define masks using Pytorch Geometric's helper functions: [here](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/mask.html). 

In [ ]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
labels = sorted_df[["int_overall"]]
labels.head()

In [ ]:
# Convert to numpy
y = labels.to_numpy()
y.shape # [num_nodes, 1] --> node regression

`Step 6`: Extract the edges

That's probably the trickiest part with a tabular dataset. You need to think of a reasonable way to connect your nodes. As mentioned previously, we will use the team assignment here.


> Reminder: There are many ways to connect the entities in a dataset and this approach is very trivial (as it will lead to disconnected subgraphs). If I wanted to build a real model from this dataset, I would probably look for a more sophisticated way to connect the players. Using a GNN is a bit overkill for the way I model the edges.


We now need to find the pairs of players that are assigned to the same team.
Let's first check how many players per team we have.

In [ ]:
# Remap player IDs
fifa_df["int_player_id"] = fifa_df.reset_index().index

In [ ]:
fifa_df.head()

In [ ]:
# This tells us how many players per team we have to connect
fifa_df["str_team_name"].value_counts()

We now need to build all permutations of these players within one team, which corresponds to a fully-connected graph within each team-subgroup. We use the column int_player_id as indices for the edges. If there is for example a [0, 1] in the edge index, it means that the first and second node (regarding the previously defined node feature matrix) are connected.


In [ ]:
import itertools
import numpy as np

teams = fifa_df["str_team_name"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))
for team in teams:
    team_df = fifa_df[fifa_df["str_team_name"] == team]
    players = team_df["int_player_id"].values
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(players, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    team_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, team_edges])
# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]

The result are these source/target edge pairs. Here you can also model dircted or undirected edges by inluding both or just one direction (I included both). This COO format is usually chosen as it is more efficient than a *NxN* adjacency matrix.

`Step 7`: Build the dataset

Now we have all the components we need to build a graph for libraries like Pytorch Geometric or DGL. \

We need to pass the numpy arrays to the Data object as follows. If you have further attributes like edge_features, you can also pass them here.
```
from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)
```

This data object represents one single graph.

Typically several graphs are combined in a dataset object. For this please refer to [the documentation](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html).

You can also quickly build a dataloader as follows. Just create a list of all your graphs and pass them to the Pytorch Geometric dataloader.

```
from torch_geometric.loader import DataLoader
data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)
```





## 1.2 Heterogeneous

`Example 2 / Step 3`

Recommender systems are a classical example for for heterogeneous graphs: [Anime Recommender Database](https://github.com/Mayank-Bhatia/Anime-Recommender) (a movie recommendation dataset).


In [ ]:
import pandas as pd

# Download data (quietly)
!wget -q https://raw.githubusercontent.com/Mayank-Bhatia/Anime-Recommender/master/data/anime.csv
!wget -q https://raw.githubusercontent.com/Mayank-Bhatia/Anime-Recommender/master/data/rating.csv

anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")

We have two tables - one that contains information about the movies (anime.csv) and another one (rating.csv) that describes how the users rated the movies.

In [ ]:
anime.head()

In [ ]:
rating.head()

Identify the graph entities we need.

- `Nodes` - Users and Animes (two node types with different features = heterogeneous)
- `Edges` - If a user has rated a movie / the rating (edge weight)
- `Node Features` - The movie attributes and for the users we have no explicit features so we have to figure something out later 
- `Labels` - The rating for a movie (link prediction regression task)

This dataset will have one single graph that contains all nodes and edges. Given a pair of node and anime movie, we will be able to predict if / how the user likes this movie. To model this as a graph, we will have to support two node types: movie and user. That's because they have different node features (and shapes) that would not fit into one joint node feature matrix.


`Preprocessing one single graph...`

`Step 4`: Extract the node features

Each of the movies occurs only once in the anime dataframe and hence we can directly extract the features from there. If you have multiple entries for each node (movie ID) in your dataframe review the remarks from 1.1. above.

We will just extract the columns with specific attributes and convert them to numeric features...

For the anime movies ...
- First we need to do a re-mapping of the IDs. That's because they don't start with 0 and also not all IDs are present. That's however important because the edge_index is always referring to the index in the node feature matrix
- We will store this mapping because we will need it later

In [ ]:
# Sort to define the order of nodes
sorted_df = anime.sort_values(by="anime_id").set_index("anime_id")

# Map IDs to start from 0
sorted_df = sorted_df.reset_index(drop=False)
movie_id_mapping = sorted_df["anime_id"]

# Select node features
node_features = sorted_df[["type", "genre", "episodes"]]
# Convert non-numeric columns
pd.set_option('mode.chained_assignment', None)

# For simplicity I'll just select the first genre here and ignore the others
genres = node_features["genre"].str.split(",", expand=True)
node_features["main_genre"] = genres[0]

# One-hot encoding
anime_node_features = pd.concat([node_features, pd.get_dummies(node_features["main_genre"])], axis=1, join='inner')
anime_node_features = pd.concat([anime_node_features, pd.get_dummies(anime_node_features["type"])], axis=1, join='inner')
anime_node_features.drop(["genre", "main_genre"], axis=1, inplace=True)
anime_node_features.head(10)

In [ ]:
# Convert to numpy
x = anime_node_features.to_numpy()
x.shape # [num_movie_nodes x movie_node_feature_dim]

For the users ...

Here we are missing a dataframe that describes the attributes of each user. As a workaround we have different options:
- Either we insert dummies (for example random values between 0 and 1 like [0, 0.5, 0.1, 1]), which will then be updated through message passing 
- Or we calculate some stats about the users, such as average rating, number of ratings, ... (based on the rating dataframe)
- Or we use typical characteristics of each node as features (degree, neighborhood, or even Node2Vec embedding)


We will go with the second option here.

```
Important: If you calculate statistics based on the dataframe, always make sure to first split into test and train set 
in order to leak no information into the dataset! (I didn't do this here)
```



In [ ]:
# Find out mean rating and number of ratings per user
mean_rating = rating.groupby("user_id")["rating"].mean().rename("mean")
num_rating = rating.groupby("user_id")["rating"].count().rename("count")
user_node_features = pd.concat([mean_rating, num_rating], axis=1)

# Remap user ID (to start at 0)
user_node_features = user_node_features.reset_index(drop=False)
user_id_mapping = user_node_features["user_id"]

# Only keep features 
user_node_features = user_node_features[["mean", "count"]]
user_node_features.head()

In [ ]:
# Convert to numpy
x = user_node_features.to_numpy()
x.shape # [num_user_nodes x user_node_feature_dim]

Those are already our node feature matrices. We could of course also normalize the values to be in the range of (0,1). 

For the movies we have clear attributes that describe each node. For the users we have calculated some basic properties that provide information about the rating behavior.

The number of nodes and the ordering is implicitly defined by their shape. Each row corresponds to one node in our final graph. 

`Step 5`: Extract the labels

In this example, we have a link prediction problem and thus the labels are the edges. The plot below shows the distribution of the ratings. Later the task will be to predict the ratings between a user and movie.

Unlike in `Example 1` the labels are now equal to the number of edges. 



In [ ]:
rating.head()

In [ ]:
# -1 means the user watched but didn't assign a weight
rating["rating"].hist()

As shown below, we don't have all of the movies in the rating table (which is natural, because we usually don't have ratings for all items). This means, we don't have labels for all user-item pairs, but only a subset. 

To consider this in the loss calculation, we can simply store a mask of the indices that are available. We just want to perform predictions for the entities for which we have a label. 

As we have an edge-prediction problem here, we implicitly stored this mask already as edge_index. For each edge we know the label and therefore we only have to calculate the loss based on the edges we know. Later at inference time, we can also predict the edge attributes (labels) for other node pairs.

In [ ]:
# Movies that are part of our rating matrix
rating["anime_id"].unique()

In [ ]:
# All movie IDs (e.g. no rating above for 1, 5, 6...)
anime["anime_id"].sort_values().unique()

In [ ]:
# We can also see that there are some movies in the rating matrix, for which we have no features (we will drop them here)
print(set(rating["anime_id"].unique()) - set(anime["anime_id"].unique()))
rating = rating[~rating["anime_id"].isin([30913, 30924, 20261])]

In [ ]:
# Extract labels
labels = rating["rating"]
labels.tail()

In [ ]:
# Convert to numpy
y = labels.to_numpy()
y.shape

`Step 6`: Extract the edges

In this example, the edges are already implicitly provided by the rating matrix. The important part however is that we need to use the remappings from before to align the IDs of the dataframes.

For each entry in the matrix, we have exactly one edge, between the user_id and the anime_id. Therefore, the edge index is exactly the same as what we have calculated in the cell above. 

The edge index can later be used in the model to mask out all edges for which we have no information.

In [ ]:
print("Before remapping...")
rating.head()

In [ ]:
# Map anime IDs 
movie_map = movie_id_mapping.reset_index().set_index("anime_id").to_dict()
rating["anime_id"] = rating["anime_id"].map(movie_map["index"]).astype(int)
# Map user IDs
user_map = user_id_mapping.reset_index().set_index("user_id").to_dict()
rating["user_id"] = rating["user_id"].map(user_map["index"]).astype(int)

print("After remapping...")
rating.head()

In [ ]:
edge_index = rating[["user_id", "anime_id"]].values.transpose()
edge_index # [2 x num_edges] 

`Step 7`: Build the dataset

Now we have all the components we need to build a graph for libraries like Pytorch Geometric or DGL. 

For Heterogenous Graphs we need to store the individual matrices in `HeteroData` objects, which can hold multiple node/edge matrices. There is also a great tutorial for [heterogenous graphs in PyG](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html#creating-heterogeneous-graphs)

```
from torch_geometric.data import HeteroData
data = HeteroData()
data['user'].x = user_node_features
data['movie'].x = anime_node_features
```

If you have different edge types between the nodes, you can also consider this here. In the example above we only have one type, therefore the edge_index looks like this (a triplet):

```
data['user', 'rating', movie'].edge_index = edge_index
```
Finally, we can add the labels of the link-prediction setup like this. In Heterogenous graphs you can also have different labels between different entities, but here we just have one type. If you build recommender systems specifically, then you might also find this tutorial on [bipartite grphs](https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html#bipartite-graphs) helpful.


```
data['user', 'movie'].y = y
```

For more information on how to handle heterogenous graphs, please refer to the documentation linked above.


# 2. Tabular dataset -> Temporal Graph dataset

In your tabular dataset identify the following:

- Nodes (Items, People, Locations, ...)
- Edges (Connections, Interactions, Correlations, ...)
- Node Features (Attributes)
- Labels (Node-level, edge-level, graph-level)
- Timesteps (Are they already defined? What is one timestep?) --> e.g. 60 min Interval
- Temporal graph shape: static or dynamic? / What is changing over time?

and optionally:
- Edge weights (Strength of the connection, number of interactions, ...)
- Edge features (Additional properties describing the edge)



`Example 3 / Step 3`

The dataset contains the [trip history of bikers in New York City](https://ride.citibikenyc.com/system-data). Luckily the start end end locations already have an ID, otherwise we would have to map the addresses to locations. 


In [ ]:
import pandas as pd
!wget -q http://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip
!unzip -o 201306-citibike-tripdata.zip

trips = pd.read_csv("201306-citibike-tripdata.csv")  
trips.head()

In [ ]:
# Remove columns we don't need / rows with Nan
cols_to_drop = ["start station name", "end station name", ]
trips.dropna(inplace=True)
trips.drop(cols_to_drop, axis=1, inplace=True)
trips.head()

In [ ]:
# Reassign the location IDs (makes it easier later, because here the IDs didn't start at 0)
locations = trips['start station id'].unique()
new_ids = list(range(len(trips['start station id'].unique())))
mapping = dict(zip(locations, new_ids))

trips['start station id'] = trips['start station id'].map(mapping)
trips['end station id'] = trips['end station id'].map(mapping)
trips.head()

**How could we use this dataset?**

For example, we could predict the trip duration between two locations, based on the surrounding (structural) and temporal trip durations. Another possibility is to predict the bike traffic in terms of how many bikers we expect on a route. 

- `Nodes` - The locations between which the bikers can travel
- `Node Features` - Attributes about a location, for example average number of bikers that start / end here, average traffic, ... (whatever is available)

For this dataset, we don't have edge (adjacency) information yet. This is often the difficult part with graph datasets. For traffic networks, you typically connect the nodes according to their closeness or even the underlying street network, i.e. if there is a direct road between two locations. Here we have the latitude/longitude for each of the addresses and with that we are able to calculate the distances. 

- `Edges / Edge weights` - Proximity between two addresses (available) / Connection according to road network (not available here)
- `Edge features` - To incorporate the individual bike information, we can simply model this as edge features i.e. number of bikes, average trip duration, ...
- `Labels` - The trip duration, which makes it a link-prediction task 

The more or less difficult part with temporal graph datasets, is to define a stepsize for the temporal snapshots. In the dataset above we have random trips from one location to another, without a pre-defined discrete interval. 
An easy approach is for example to define X minute intervals and build a graph out of all trips that happened (ended) in this timeframe. The "ended" part is important, because we don't want to leak information into the future timesteps.

- `Timesteps` - 60 min steps (we have 1 month of data = ~ 700 graphs)



`Building the temporal dataset ...`

First make sure the dataset is sorted in time, so that we can iterate over the 60-min intervals. 

In [ ]:
trips = trips.sort_values(by="starttime")
trips.head()

Now we want to iterate over the dataframe and select all trips that fall into each of the 60 min intervals. 

Just a quick check - how many trips per bucket will we end up with in the end? This defines for how many edges we have information in each individual graph


In [ ]:
from datetime import datetime, timedelta
import seaborn as sns
sns.set(rc={'figure.figsize':(20,6)})

# Convert columns to datetime
trips["starttime"] = pd.to_datetime(trips["starttime"], format="%Y-%m-%d %H:%M:%S")
trips["stoptime"] = pd.to_datetime(trips["stoptime"], format="%Y-%m-%d %H:%M:%S")

start_date = datetime.strptime("2013-06-01 00:00:01", "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime("2013-07-01 00:10:34", "%Y-%m-%d %H:%M:%S")
interval = timedelta(minutes=60)
bucket_elements = []
while start_date <= end_date:
    # Check how many trips fall into this interval
    bucket_elements.append(trips[((start_date + interval) >= trips["stoptime"])
                                  & (start_date <= trips["stoptime"])].shape[0])
    # Increment
    start_date += interval

sns.scatterplot(x="index", y="trips_per_hour", data=pd.DataFrame(bucket_elements, columns=["trips_per_hour"]).reset_index())

Chosing the right interval-size is an important design decision in temporal models. If it's too large, we will aggregate to much information and lose the fine grained information over time. If it's too small, we have a lot of noise in the data and the model might not be able to identify clear patterns. 

We can see a clear trend here - at night there are way less bikers on the streets. Also, it seems that there is a global trend, which results in more and more bikers in total over time (probably because the weather is getting warmer). 

So is this the right step-size? We can't really say yet - we have to try it out. But what we can see is that the number of bikers slowly changes over the day and therefore we probably don't aggregate too much information. Feel free to play around with the stepsize above - enter for example 600 (= 10 hours) or 10 (= 10 mins). Eventually it's also important to consider how far in the future we want to predict later.

`Step 4`: Extract the node features


**What type of temporal GNN dataset do I have?**

Another important point is how our graph changes over time. It can happen, that not all nodes are present in the next time-step, which makes it a bit difficult as our edge indices are shifted. Below I have some hints on how to cope with each of the graph types. 

`Static Graph Temporal Signal` 

This is simply a graph that always stays the same and only the label information changes. Common examples are road networks (the nodes/edges do not suddently disappear in a long-range time horizon) or electricity networks. 
For this we can either use [pytorch geometric temporal](https://pytorch-geometric-temporal.readthedocs.io/en/latest/notes/installation.html) or also only [pytorch geometric](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.TemporalData), which recently added temporal graph support (but only dynamic edges).

This is by far the easiest graph type, because the node feature matrix and edge_index stay untouched. We only have to adjust the labels in each graph snapshot.

`Dynamic Graph Temporal/Static Signal`

The more difficult graph type is a dynamic graph (with regards to nodes/edges). 
This typically happens in social networks, that quickly grow/shrink over time, but also transaction systems like crypto networks.
Previously I mentioned that the node ordering is implicitly defined by the node feature matrix - but what if this matrix changes. 

`Option 1` 

The easiest solution is just to append new nodes (in it's temporal order at preprocessing-time) to this matrix and always use the full matrix (all nodes) even if not all of them are part of the current snapshot. This matrix can (depending on your nodes) get very big. Therefore I suggest to check what your maximum number of nodes is and if it's feasible to store your total node_feature matrix in memory. 
If your node_features change over time, that's also no problem - you can simply update the node feature matrix for each snapshot. The important point for this option is that the ordering of the nodes always stays the same! This means index 0 is always the same node (e.g. location), even if it's not really used in this snapshot.


`Option 2` 

Option 1 will lead to a very big matrix with a lot of redundancies - but has one advantage: You always have the same index for a specific node. It can happen that a single node only occurs once and then is carried through all other snapshots, even if it's never used again. If you want to make your graph *really* dynamic, you should also be able to remove nodes. This however will affect the edge_index, because the indices of the node_feature matrix change!
Therefore we would need to re-index the edge_index to point to the correct nodes. But besides this there is another issue.

In spatio-temporal GNN models, you typically update the node feature embeddings over time (with a recurrent unit). The problem really is that you need to update nodes between two snapshots, based on their indices -> But what if a node is not present anymore. So as you can see it's not that trivial to learn on a dynamic graph. Therefore I'd suggest to make the first Option work. :)


---

In our case we have a `Static Graph Temporal Signal`, because our locations/edges do not really change over time. But as we will see later, when incorporating the labels as edge_features (for historical snapshots), we will end up with a `Dynamic Graph Temporal Signal`. 


> Coming back to the node features...

We don't really have information about the nodes (locations) here but we need them in order to apply message passing. If really nothing comes to your mind, I would probably assign random values (gaussian samples) to the node feature matrix.

Here we can model the outgoing / incoming traffic as an attribute for the nodes. This allows us to say "this is a heavily frequented location" or not. 
We can calculate this information based on the dataset, but here it's always important to consider that we should apply a train/test split in advance (which I didn't do here) to leak no information into the train set. 


In [ ]:
import numpy as np
# Find out how many outgoing bikers we have
outgoing_trips = trips.groupby("start station id").count()["bikeid"].values
incoming_trips = trips.groupby("end station id").count()["bikeid"].values

# Normalize features between 0 and 1
outgoing_trips = (outgoing_trips - np.min(outgoing_trips)) / (np.max(outgoing_trips) - np.min(outgoing_trips))
incoming_trips = (incoming_trips - np.min(incoming_trips)) / (np.max(incoming_trips) - np.min(incoming_trips))

# Build node features
node_features = np.stack([outgoing_trips, incoming_trips]).transpose()
print("Full shape: ", node_features.shape)
node_features[:10] # [num_nodes x num_features]

That's a simple example for a possible node feature matrix. As already mentioned - you can be creative here. :)

`Step 5`: Extract the edges

As mentioned previously, we want to create the edges based on the distances of the locations, as this is all information we have in this dataset. The hope is, that the GNN can later identify "crowded" places, where the tripduration will most likely take longer (due to traffic).

The topology of our graph is therefore **static** in this dataset, because the edges don't change and also the nodes are always the same. Therefore we can pre-compute this part of the graph. In fact, the only temporal part here are the labels (tripdurations), as we will see further below. 

Therefore, we only calculate the edges once here for all snapshots. This can be efficiently done by building all combinations (the cartesian product) and applying the distance calculation on each row. **Remember: when dealing with dataframes there is almost always a better way than using a for-loop** 

But there was one problem I encountered during modelling the labels. I planned to use past labels (aka historical trip durations) as edge features, as this will add a lot of information to the graph. This basically tells us "this is the current trip duration for some parts of the graph (edge features), please predict 1 hour into the future for other parts of the graph" (labels).
Edge features however can only "live" on existing edges and unfortunately the static edges here (based on the location) are different from the bike-trips (which are not distance-based). Because of that I will extend the edge index later by the edges for the bike trips and will use edge features to signal that these added edges have different properties than the once I add here.
Consequentely, we will use a `Dynamic Graph Temporal Signal` later.
Note that this situation comes from the fact that we have two edges types - one for the distance-based edges and one for the trips. We could also model this as a heterogeneous graph instead of providing the information as edge features.


> Edge feats: [distance, edge_type, historical trip duration]





In [ ]:
from sklearn.utils.extmath import cartesian
from geopy.distance import geodesic

# Get all possible start locations and their geo info
subset = ["start station longitude", "start station latitude", "start station id"]
all_starts = trips.drop_duplicates(subset="start station id", keep="first")[subset]
# Get all possible end locations and their geo info
subset = ["end station longitude", "end station latitude", "end station id"]
all_ends = trips.drop_duplicates(subset="end station id", keep="first")[subset]
# Combine all combinations in one dataframe
distance_matrix = all_ends.merge(all_starts, how="cross")
distance_matrix["distance"] = distance_matrix.apply(lambda x: geodesic((x["start station latitude"], x["start station longitude"]), 
                                                          (x["end station latitude"], x["end station longitude"])).meters, axis=1)
distance_matrix.head()

Based on the new "distance" column we can now create edges. For this we can select a threshold to only connect nodes that are close to each other. Alternatively, you can also take all edges and assign a weight to each edge in the graph.

The distance column is based on meters and here I'll just use 500 meteres as a cutoff (simply plot a histogram to find a good threshold). This will also generate self-loops (if you dont want that, simply also add > X).

In [ ]:
distance_matrix["edge"] = distance_matrix["distance"] < 500
distance_matrix.head()

Now we are almost there! We just need a way to build the edge_index. For this, we need to consider the original ordering in the node feature matrix. Remember when we did the mapping of the indices before? - Because of that we already have the edge indices set.

In [ ]:
# Use mask to extract static edges
edge_index = distance_matrix[distance_matrix["edge"] == True][["start station id", "end station id"]].values
edge_index = edge_index.transpose()
edge_index # [2 x num_edges]

In [ ]:
# Add edge features to indicate edge type
distance_feature = distance_matrix[distance_matrix["edge"] == True]["distance"].values
edge_type_feature = np.zeros_like(distance_feature) # 0 = static edge
trip_duration_feature = np.zeros_like(distance_feature) # 0 = no information
static_edge_features = np.stack([distance_feature, edge_type_feature, trip_duration_feature]).transpose()
static_edge_features # [num_edges x num_features]

`Step 6`: Extract the labels and build the dataset

Now we get in touch with the temporal aspect of our dataset. Because the edge_index and the node features were static, we could pre-compute them for each snapshot. The labels however change over time, because we have different trip durations for different bikers for each 60 min interval. In addition to that we can also use historical labels as edge features. This is a special attribute of time-series datasets - the labels become features for past timesteps. If you have a node-level prediction task you can add the historical labels as node features to each of the nodes. But this also means that you cannot pre-compute the node feature matrix and need to do it in the following loop instead.

> In most of the time-series libraries this is called a "lag" or "offset". In our dataset we want to predict 1 hour into the future, based on the current situation (trip durations). Therefore, we can use the current trip durations as edge features and the trip durations of the next snapshot as targets. Of course you could also define a larger offset, for example 12 hours into the future.

As mentioned previously, we have to types of edges now - static edges based on the location and edges for historical trips (for which we need to insert new edges to use the edge features). It can happen that we have multiple bikers on the same "edge" (=route) - here we simply average all of the trip durations between two nodes.

Generally, everything that is temporal needs to be computed in a loop over the time-series. Because of that, we loop over the start and end time of our dataset and store the labels of each subsequent snapshot in a list and the current trip durations as edge features in another list.
We also stack the pre-computed node_features and edge_index, so that the first entry in each list corresponds to the first snapshot and so on. That's at least how it is typically done in [pytorch geometric temporal](https://pytorch-geometric-temporal.readthedocs.io/en/latest/index.html). Note that there is also a TemporalData Object in plain PyG available now. 



In [ ]:
def extract_dynamic_edges(s):
    # Extract dynamic edges and their features
    trip_indices = s[["start station id", "end station id"]].values
    trip_durations = s["tripduration"]

    # Build edge features
    distance_feature  = pd.DataFrame(trip_indices, 
                                    columns=["start station id", "end station id"]).merge(
                                        distance_matrix, on=["start station id", "end station id"], 
                                        how="left")["distance"].values
    edge_type_feature = np.ones_like(distance_feature) # 1 = dynamic
    trip_duration_feature = trip_durations
    edge_features = np.stack([distance_feature, edge_type_feature, trip_duration_feature]).transpose()
    return edge_features, trip_indices.transpose()



start_date = datetime.strptime("2013-06-01 00:00:01", "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime("2013-07-01 00:10:34", "%Y-%m-%d %H:%M:%S")
interval = timedelta(minutes=60)

xs = []
edge_indices = []
ys = []
y_indices = []
edge_features = []


while start_date <= end_date:
    # 0 - 60 min 
    current_snapshot = trips[((start_date + interval) >= trips["stoptime"])
                                  & (start_date <= trips["stoptime"])]
    # 60 - 120 min
    subsequent_snapshot = trips[((start_date + 2*interval) >= trips["stoptime"])
                                  & (start_date + interval <= trips["stoptime"])]
    # Average duplicate trips
    current_snapshot = current_snapshot.groupby(["start station id", "end station id"]).mean().reset_index()
    subsequent_snapshot = subsequent_snapshot.groupby(["start station id", "end station id"]).mean().reset_index()

    # Extract dynamic trip edges
    edge_feats, additional_edge_index = extract_dynamic_edges(current_snapshot)
    exteneded_edge_index = np.concatenate([edge_index, additional_edge_index], axis=1)
    extended_edge_feats = np.concatenate([edge_feats, static_edge_features], axis=0)

    # Labels
    y = subsequent_snapshot["tripduration"].values
    y_index = subsequent_snapshot[["start station id", "end station id"]].values

    # Append everything
    xs.append(node_features) # static
    edge_indices.append(exteneded_edge_index) # static + dynamic
    edge_features.append(extended_edge_feats) # static + dynamic
    ys.append(y) # dynamic
    y_indices.append(y_index.transpose()) # dynamic

    # Increment
    start_date += interval

Ok so what did we just do here? 

After each 60 min interval, we get a subset of our dataframe for this time range. Based on that, we extract all the available edge labels between two locations. Those are the ones we want to predict later in the model.
In order to calculate the loss only based on the edges for which we have labels, we store some sort of mask (y_index) that tells us for which source/target pairs we have labels. 

> Important: Here we take the label of each snapshot as target value. 

We could of course also normalize the labels to ensure smoother training for the regression setup. 


**Now we have everything we need! ** 

In [ ]:
i = 2
print(f"""Example of graph snapshot {i}: \n
      Node feature shape: {xs[i].shape} \n
      Edge index shape: {edge_indices[i].shape} \n
      Edge feature shape: {edge_features[i].shape} \n 
      Labels shape: {ys[i].shape} \n
      Labels mask shape: {y_indices[i].shape}
      """)

Just like before, I won't install Pytorch Geometric Temporal here, as this will make the notebook too heavy, but here are some code snippets for the final steps.


We need to pass the lists of numpy arrays to the Data Structures, as for example done [here](https://pytorch-geometric-temporal.readthedocs.io/en/latest/_modules/torch_geometric_temporal/dataset/wikimath.html#WikiMathsDatasetLoader).

```
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
dataset = DynamicGraphTemporalSignal(
            edge_indices, edge_features, xs, ys, y_indices=y_indices
        )

```

For more details please have a look [at the documentation](https://pytorch-geometric-temporal.readthedocs.io/en/latest/modules/signal.html).
